# 第11回講義 宿題

## 課題. Attentionを用いたキャプション生成モデルを実装せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データのtrain_X, train_yのみが与えられます
    - train_Xとtrain_yをtrain_X, valid_xとtrain_y, valid_yに分けるなどしてモデルを学習させてください
    - **cost関数を戻り値**としてください
- **test_X, test_yに対する交差エントロピー(負の対数尤度)の平均で評価**します
- **lecture_chap11_imgcap_exercise_pub.ipynb**でダウンロードしたデータを使用します
- 全体の実行時間がiLect上で120分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [ ]:
def homework(train_X, train_y):
    global vocab_size, sess, x, d # 下記Checker Cell内 で定義した各変数を利用可能.
    
    # WRITE ME!
    #  Hint: 下記Checker Cell内でimportした pad_sequences, VGG16 は利用可能.
    
    return cost # 返り値のcostは,tensorflowの計算グラフのcostを返す.

# Checker Cell (for student)

In [ ]:
import sys

from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.applications.vgg16 import VGG16
rng = np.random.RandomState(42)
random_state = 42

del [
    tf.app,
    tf.compat,
    tf.contrib,
    tf.errors,
    tf.gfile,
    tf.graph_util,
    tf.image,
    tf.layers,
    tf.logging,
    tf.losses,
    tf.metrics,
    tf.python_io,
    tf.resource_loader,
    tf.saved_model,
    tf.sdca,
    tf.sets,
    tf.summary,
    tf.sysconfig,
    tf.test
]

sys.modules['keras'] = None

def build_vocab(file_path, target):
    vocab = set()
    for line in open(file_path, encoding='utf-8'):
        words = line.strip().split()
        vocab.update(words)

    if target:
        w2i = {w: np.int32(i+2) for i, w in enumerate(vocab)}
        w2i['<s>'], w2i['</s>'] = np.int32(0), np.int32(1)
    else:
        w2i = {w: np.int32(i) for i, w in enumerate(vocab)}

    return w2i

def encode(sentence, w2i):
    encoded_sentence = []
    for w in sentence:
        encoded_sentence.append(w2i[w])
    return encoded_sentence

def load_data(file_path, vocab=None, w2i=None, target=True):
    if vocab is None and w2i is None:
        w2i = build_vocab(file_path, target)
    
    data = []
    for line in open(file_path, encoding='utf-8'):
        s = line.strip().split()
        if target:
            s = ['<s>'] + s + ['</s>']
        enc = encode(s, w2i)
        data.append(enc)
    i2w = {i: w for w, i in w2i.items()}
    return data, w2i, i2w

def validate_homework():
    global vocab_size, sess, x, d
    
    train_y, w2i, i2w = load_data('./mscoco_captions_10000.txt', target=True)
    train_X = np.load('./mscoco_images_10000.npy')
    
    vocab_size = len(i2w)
    
    train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=42)
    
    # validate for small dataset
    train_X_mini = train_X[:100]
    train_y_mini = train_y[:100]
    test_X_mini = test_X[:100]
    test_y_mini = test_y[:100]
    
    tf.reset_default_graph()

    with tf.Session() as sess:
        x = tf.placeholder(tf.float32, [None, 224, 224, 3])
        d = tf.placeholder(tf.int32, [None, None], name='d')

        cost = homework(train_X_mini, train_y_mini)
        # sess, x, d はグローバル変数で定義されているので homework関数内で利用可能.
        # 返り値のcostは,tensorflowの計算グラフのcostを返す.
        
        test_batch_size = 32
        n_batches_test = -(-len(test_X_mini) // test_batch_size)
        test_costs = []

        for i in range(n_batches_test):
            start = i * test_batch_size
            end = start + test_batch_size if (start + test_batch_size) < len(test_X_mini) else len(test_X_mini)

            test_X_mb = test_X_mini[start:end]
            test_y_mb = np.array(pad_sequences(test_y_mini[start:end], padding='post', value=-1))

            test_cost = sess.run(cost, feed_dict={x: test_X_mb, d: test_y_mb})

            test_costs.append(test_cost)
            
    print(np.mean(test_costs))

def score_homework():
    global vocab_size, sess, x, d
    
    train_y, w2i, i2w = load_data('./mscoco_captions_10000.txt', target=True)
    train_X = np.load('./mscoco_images_10000.npy')
    
    vocab_size = len(i2w)
    
    train_X, test_X, train_y, test_y = train_test_split(train_X, train_y, test_size=0.1, random_state=42)
    
    tf.reset_default_graph()

    with tf.Session() as sess:
        x = tf.placeholder(tf.float32, [None, 224, 224, 3])
        d = tf.placeholder(tf.int32, [None, None], name='d')

        cost = homework(train_X, train_y)
        # sess, x, d はグローバル変数で定義されているので homework関数内で利用可能.
        # 返り値のcostは,tensorflowの計算グラフのcostを返す.
        
        test_batch_size = 32
        n_batches_test = -(-len(test_X) // test_batch_size)
        test_costs = []

        for i in range(n_batches_test):
            start = i * test_batch_size
            end = start + test_batch_size if (start + test_batch_size) < len(test_X) else len(test_X)

            test_X_mb = test_X[start:end]
            test_y_mb = np.array(pad_sequences(test_y[start:end], padding='post', value=-1))

            test_cost = sess.run(cost, feed_dict={x: test_X_mb, d: test_y_mb})

            test_costs.append(test_cost)
            
    print(np.mean(test_costs))

In [ ]:
validate_homework()
# score_homework()